#### Region

In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail_migration.silver.region (
  r_regionkey INT,
  r_name STRING,
  r_comment STRING,
  CONSTRAINT pk_region PRIMARY KEY (r_regionkey)
)
USING DELTA;

INSERT OVERWRITE retail_migration.silver.region
SELECT * FROM retail_migration.bronze.region;

#### Nation

In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail_migration.silver.nation (
  n_nationkey INT,
  n_name STRING,
  n_regionkey INT,
  n_comment STRING,
  CONSTRAINT pk_nation PRIMARY KEY (n_nationkey),
  CONSTRAINT fk_nation_region FOREIGN KEY (n_regionkey)
    REFERENCES retail_migration.silver.region (r_regionkey)
)
USING DELTA;

INSERT OVERWRITE retail_migration.silver.nation
SELECT * FROM retail_migration.bronze.nation;


#### Customer

In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail_migration.silver.customer (
  c_custkey INT,
  c_name STRING,
  c_address STRING,
  c_nationkey INT,
  c_phone STRING,
  c_acctbal DOUBLE,
  c_mktsegment STRING,
  c_comment STRING,
  CONSTRAINT pk_customer PRIMARY KEY (c_custkey),
  CONSTRAINT fk_customer_nation FOREIGN KEY (c_nationkey)
    REFERENCES retail_migration.silver.nation (n_nationkey)
)
USING DELTA;

INSERT OVERWRITE retail_migration.silver.customer
SELECT * FROM retail_migration.bronze.customer;


#### Supplier

In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail_migration.silver.supplier (
  s_suppkey INT,
  s_name STRING,
  s_address STRING,
  s_nationkey INT,
  s_phone STRING,
  s_acctbal DOUBLE,
  s_comment STRING,
  CONSTRAINT pk_supplier PRIMARY KEY (s_suppkey),
  CONSTRAINT fk_supplier_nation FOREIGN KEY (s_nationkey)
    REFERENCES retail_migration.silver.nation (n_nationkey)
)
USING DELTA;

INSERT OVERWRITE retail_migration.silver.supplier
SELECT * FROM retail_migration.bronze.supplier;


#### Part

In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail_migration.silver.part (
  p_partkey INT,
  p_name STRING,
  p_mfgr STRING,
  p_brand STRING,
  p_type STRING,
  p_size INT,
  p_container STRING,
  p_retailprice DOUBLE,
  p_comment STRING,
  CONSTRAINT pk_part PRIMARY KEY (p_partkey)
)
USING DELTA;

INSERT OVERWRITE retail_migration.silver.part
SELECT * FROM retail_migration.bronze.part;


#### Partsupp

In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail_migration.silver.partsupp (
  ps_partkey INT,
  ps_suppkey INT,
  ps_availqty INT,
  ps_supplycost DOUBLE,
  ps_comment STRING,
  CONSTRAINT pk_partsupp PRIMARY KEY (ps_partkey, ps_suppkey),
  CONSTRAINT fk_partsupp_part FOREIGN KEY (ps_partkey)
    REFERENCES retail_migration.silver.part (p_partkey),
  CONSTRAINT fk_partsupp_supplier FOREIGN KEY (ps_suppkey)
    REFERENCES retail_migration.silver.supplier (s_suppkey)
)
USING DELTA;

INSERT OVERWRITE retail_migration.silver.partsupp
SELECT * FROM retail_migration.bronze.partsupp;


#### Orders

In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail_migration.silver.orders (
  o_orderkey INT,
  o_custkey INT,
  o_orderstatus STRING,
  o_totalprice DOUBLE,
  o_orderdate DATE,
  o_orderpriority STRING,
  o_clerk STRING,
  o_shippriority INT,
  o_comment STRING,
  CONSTRAINT pk_orders PRIMARY KEY (o_orderkey),
  CONSTRAINT fk_orders_customer FOREIGN KEY (o_custkey)
    REFERENCES retail_migration.silver.customer (c_custkey)
)
USING DELTA;

INSERT OVERWRITE retail_migration.silver.orders
SELECT * FROM retail_migration.bronze.orders;


#### Lineitem

In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail_migration.silver.lineitem (
  l_orderkey INT,
  l_partkey INT,
  l_suppkey INT,
  l_linenumber INT,
  l_quantity DOUBLE,
  l_extendedprice DOUBLE,
  l_discount DOUBLE,
  l_tax DOUBLE,
  l_returnflag STRING,
  l_linestatus STRING,
  l_shipdate DATE,
  l_commitdate DATE,
  l_receiptdate DATE,
  l_shipinstruct STRING,
  l_shipmode STRING,
  l_comment STRING,
  CONSTRAINT pk_lineitem PRIMARY KEY (l_orderkey, l_linenumber),
  CONSTRAINT fk_lineitem_orders FOREIGN KEY (l_orderkey)
    REFERENCES retail_migration.silver.orders (o_orderkey),
  CONSTRAINT fk_lineitem_part FOREIGN KEY (l_partkey)
    REFERENCES retail_migration.silver.part (p_partkey),
  CONSTRAINT fk_lineitem_supplier FOREIGN KEY (l_suppkey)
    REFERENCES retail_migration.silver.supplier (s_suppkey)
)
USING DELTA;

INSERT OVERWRITE retail_migration.silver.lineitem
SELECT * FROM retail_migration.bronze.lineitem;


#### Validation

In [0]:
source_catalog = "samples.tpch"

In [0]:
tables = [t.name for t in spark.catalog.listTables(source_catalog) if not t.name.startswith("_")]
print(tables)

In [0]:
for t in tables:
    bronze_count = spark.table(f"retail_migration.bronze.{t}").count()
    silver_count = spark.table(f"retail_migration.silver.{t}").count()
    print(t, bronze_count == silver_count)

**As we see all rows where copied correctly**

In [0]:
%sql
USE retail_migration.silver;
SELECT current_catalog(), current_schema();


In [0]:
spark.conf.get("spark.databricks")